# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
input_keyword = "財經"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                                    # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))           # 擷取中間的編碼結果
ETtoday_url = 'https://www.ettoday.net/news/focus/' + utf8_url[2:-1:1] +'/'
print(ETtoday_url)

財經: %E8%B2%A1%E7%B6%93
https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/


In [3]:
#ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

提示：可參考以下的Stack Overflow: 
https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-in-dynamically-loading-webpage/48851166#48851166

In [4]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

# 以下是用Selenium模擬下拉網頁動作，讓網頁更新

last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")  #滾動到頁面底部

while True:
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(SCROLL_PAUSE_TIME)
    
    new_height = browser.execute_script('return document.body.scrollHeight')
    
    if new_height == last_height:
        print('到達頁面底端')
        break
    
    last_height = new_height
    print('網頁更新中...')

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。

def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [ ]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):     #enumerate自動加入索引，http://book.pythontips.com/en/latest/enumerate.html
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    
    # 連到外部連結，擷取詳細新聞內容
    
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 疫情下各國大力救市　投資人善用4心法尋求抄底時機

▲新冠肺炎疫情在各國蔓延，投資人可善用4心法尋求抄底時機。（示意圖／路透社）
記者紀佳妘／台北報導
新冠肺炎（COVID-19）衝擊全球金融市場，投資人關注市場是否已經觸底，以及遭疫情打壓的股票是否值得逢低吸納，瑞銀（UBS）對此提出4點建議，投資人可以開始分批逢低進場。
瑞銀財富管理亞太區首席投資總監陳敏蘭表示，隨著全球努力控制疫情，並緩解經濟衝擊，風險資產仍將繼續震盪，若各國無法壓平疫情曲線，市場可能再度下探近期低點，但另一方面，由於全球推出前所未有的刺激措施，一旦市場掉頭向上，反彈幅度將相當強勁。
面對疫情來勢洶洶，投資人關注後續布局方向，陳敏蘭提出4點建議，首先是利用偏高的波動率，全球市場目前處於空頭市場，投資人可尋求將多餘的現金部署在價格出現折價的投資目標，建構頭寸的一個方法是結合分批入市和賣出選擇權賣權策略。分批買入較高風險的資產，有助於投資組合平順渡過短期震盪，而賣出賣權能帶來額外收益，並在市場進一步下跌時，預先鎖定參與的機會。
第二，為投資組合提供下行保護，陳敏蘭說，擔心市場進一步下跌的投資人，可以增加資本保護，提高高評等債券部位，或是追加在動態資產配置策略中的資本投入，投資人也可考慮增加黃金和長期美國公債的部位或投資私募市場。
第三，相對股票看好信用債，陳敏蘭認為，雖然股價反映了衰退預期，但相較於股票，信用債定價更接近上述的下行情景，因此美國投資級債券、美國高收益債券、美元計價的新興市場主權債券以及中資房企高收益短債提供了具吸引力的機會。
第四，精選股票投資，瑞銀聚焦於以下3個領域，第一是超賣的股票，疫情衝擊下股市全面受壓，即便資產負債表強勁的優質公司股價也可能大幅下跌，因此看好半導體的優質週期股和受惠於大陸房地產整合的企業。
另在相對抗跌的股票，經濟前景不明朗之際，盈餘成長具韌性的行業及公司更具吸引力；至於長期受益者，疫情危機為生活帶來許多變化，其中有些是短期改變，而疫情也加速了一些長期趨勢變化，讓新科技得到更廣泛運用，如遠端工作、虛擬學習以及線上醫療服務，這又將促進對5G、人工智慧、大數據和雲端計算等使能技術的投資。
-------------

▲內蒙古煤炭產業嚴重貪汙，大陸中央下令徹查20年來腐敗案件。（圖／翻攝內蒙古煤炭地質勘查集團官網）
記者任以芳／綜合報導
日前，大陸中央下令徹查，追溯內蒙古20年來貪汙案。根據中央紀委國家監委網站今18日最新消息指出，又拿下兩名煤炭領域幹部。內蒙古煤炭地質勘查（集團）有限責任公司前任黨委書記、總經理莫若平，以及後續接任的郝勝發，兩人涉嫌嚴重違紀違法，正在接受調查。
根據澎湃新聞指出，莫若平、郝勝發長期在內蒙古自治區煤田地質局工作。莫若平2017年10月退休。接替總經理職務的人正是郝勝發。大陸媒體有形容，內蒙古自治區正在對20年內煤炭領域違法違規問題，進行一場大起底調查。
▲內蒙古煤炭產業官商勾結20年，如今被中央盯上。（圖／翻攝內蒙古煤炭地質勘查集團官網）
今年2月28日，內蒙古自治區的煤炭行業迎來一場反腐風暴。執法單位先針對雲光中、白向群、邢雲、雲公民四人進行調查整治，發現他們長期把持煤礦產業，官商勾結等罪行，同時暴露出，內蒙古煤炭產業長期以來貪腐問題。違規違法獲取、倒賣、違法配置煤炭資源，涉煤腐敗問題嚴重污染政治生態。大陸中央也宣布，將針對2000年以來涉煤領域所有企業、所有項目、所有公職人員進行全方位排查。
該報導指出，涉煤反腐倒查時間跨度有20年之久，消息一出，引發社會各界關注。由於，內蒙古煤炭資源豐富、種類齊全優勢彰顯，各類資本開始大量在內蒙古佈局煤炭產業。從2002年至2013年，煤炭需求迅速增加，導致煤炭價格飛漲，煤炭行業進入快速發展的黃金十年。
▲一煤獨大的內蒙古嘗試轉型，徹查20年貪汙案將衝擊當地產業。（圖／翻攝內蒙古煤炭地質勘查集團官網）
不過，近年大陸經濟增速放緩，煤炭價格開始下跌，內蒙古快速跌入低潮，經濟增長迅速從前列，變成倒數。經濟轉型的壓力下，過分依賴煤炭產業結構的缺陷被放大。
內蒙古官方早就意識到這種「一煤獨大」的經濟結構的危險。如今，內蒙古自治區尚未擺脫對煤炭經濟的依賴，涉煤反腐敗倒查20年也將衝擊當地產業。未來，煤炭行業轉型與涉煤反腐敗調查能否有平衡點，值得觀察。
----------------------------------------------------------------------

[11] 三星靠5G拯救低迷銷量　外媒：Galaxy A系列5G手機將更多、更便宜

▲ 三星Galaxy S20+ 5G晴空白

▲英專家透露警訊：「我們是歷史上首個比父母更貧窮的世代」。（示意圖／《寄生上流》劇照，CatchPlay提供）
記者林育綾／綜合報導
現代許多年輕人將買房視為遙不可及的夢想，薪資漲幅更跟不上通貨膨脹。英國經濟學專家馬克．湯瑪斯（Mark Thomas）在新書《下流世代》大膽預測，如果我們再不做些什麼，2050年以前「全球經濟就會面臨崩潰」，更透露警訊「我們是歷史上首個比父母更貧窮的世代」。
商周出版近來發行英國專家馬克．湯瑪斯（Mark Thomas）的著作《下流世代》（99%: Mass Impoverishment and How We Can End It），提到「我們正活在焦慮年代，大規模貧困就在你身邊，這波災難性的走向，讓年輕世代身陷貧苦泥沼無法脫身。」
作者以服務於英國知名顧問管理公司多年的經歷，透過經濟學的角度預見一個即將發生的未來，「如果我們再不做些什麼，在2050年以前，全球經濟就會面臨崩潰。」此外他更透露了一個警訊，「我們是歷史上首個比父母更貧窮的世代」，未來甚至可能要依賴食物銀行，老年生活更成為所有人的慢性焦慮。
「如果經濟表現良好，那麼大多數人怎麼會表現不好？如果經濟大餅增長了，為什麼我們都沒有得到更大的份額？這本書顯示了我們99％的人可以採取的行動，以消除大規模貧困並建立一個值得生活的社會：一個人人受益的富裕時代。」
該書除了獲得英國亞馬遜五星好評暢銷書，如今中文版在台發行，也獲得《做工的人》作家林立青、公民教師及《思辨》作者黃益中推薦。
▲下流世代：我們注定比父母更貧窮。（圖／商周出版提供）
----------------------------------------------------------------------

[23] 瑞德西韋治癒效果「激勵美股」　道瓊工業指數開盤上漲600點

▲美股受到瑞德西韋激勵，道瓊工業指數開盤上漲600點。（圖／路透）
記者林彥臣／綜合報導
受到吉利德公司（Gilead）發表瑞德西韋治療新冠肺炎（COVID-19）效果顯著影響，美股受到利多的激勵，道瓊工業指數（Dow Jones Industrial Average Index）開盤就上漲600點。
根據CNBC報導，道瓊工業指數開盤上漲600點，漲幅超過2%，標準普爾成交量增加1.8%，那斯達克綜合（Nasdaq Composite）

▲有人上網高價哄抬醫用口罩，經濟部今（17）日表示，經查證已確認貨源之一是雲林的一家口罩廠。（圖／記者湯興漢攝）
記者林淑慧／台北報導
有媒體報導，警方查到有人上網高價哄抬醫用口罩，經濟部今（17）日表示，經查證已確認貨源之一是雲林的一家口罩廠，將全力協助檢警調查；另外，也將對該公司採減供30％熔噴不織布嚴加控管。
經濟部官員提醒，口罩為重要防疫物資，依照傳染病防治法，政府已經開始徵用的防疫物資，如果有惡意囤積或哄抬價格的行為，最高可處 7 年有期徒刑、併科罰金 500 萬元，提醒大家千萬母湯以身試法。
官員指出，案情還在調查中，但對於報導中的公司生產異常，經濟部會先減供關鍵原料熔噴不織布 30％，也會持續查核口罩廠的產量，如果還有不合理，會再調整熔噴不織布的分配跟國軍支援人力等措施。
----------------------------------------------------------------------

[35] 先探／Switch概念股人氣放閃

近期爆紅的《集合吧！動物森友會》帶動Switch遊戲機銷售長紅，為滿足市場需求，任天堂積極拉貨，為台灣供應鏈包含旺宏、原相等廠商業績添柴火。
文／吳旻蓁
受新冠肺炎疫情影響，全球經濟活動大範圍停擺，而足不出戶、社交隔離成為這陣子許多人的生活常態，也讓宅經濟商機在短時間內迅速竄紅，從最初的電子商務買氣旺盛，因遠端趨勢增加的電腦需求，到近期日本遊戲巨頭任天堂的Switch銷量暴增等，都為疫情之下宅商機的升溫提供了最佳註解。
任天堂股價水漲船高
受惠《健身環大冒險》和《集合吧！動物森友會》兩款遊戲熱銷帶動，加上早在二月初，中國生產線受封城影響，導致Switch遊戲機延後出貨，在兩大因素夾擊之下，促使Switch需求大增，全球各地都面臨到一機難求的景況，供不應求也導致二手Switch價格水漲船高，已遠超過新機原價。
問世三年的Switch銷售成績頗佳，在今年一月底公布財報時，任天堂就已經將今年度Switch系列機種全球銷售量從原本預估的一八○○萬台，上修至一九五○萬台，如今更隨著宅經濟發酵銷量暴增，市場也樂觀看好，Switch全年出貨量可望超越公司原本的預估值。
同樣水漲船高的還有任天堂股價，從三月十六日這波低點算起，近一個月，已大漲逾四成，摩根士丹利近期就將任天堂評等從持有調升至加碼，且目標價為每股五．

▲全台無薪假人數逼近1.5萬人，經濟部加碼補貼力挺企業。（圖／記者湯興漢攝）
記者林淑慧／台北報導
全台無薪假人數逼近1.5萬人，政府加碼補貼力挺企業。經濟部次長林全能今（17）日表示，經濟部已規劃工業園區費用減收緩繳，企業可緩繳1年或減租二成，減租期限為期半年，以減輕企業負擔渡過這「疫」關。
勞動部16日公布最新無薪假統計，截至15日止，全台總計588家企業實施無薪假、受影響人數高達1萬4,821人，無薪假情況直逼金融海嘯，創下2009年12月來新高。行政院今日舉行「紓困挺製造 一起度難關」記者會，由政務委員龔明鑫、經濟部次長林全能出席說明最新製造業紓困方案。
針對工業區相關費用減收緩繳，林全能指出，提供經濟部工業局轄管工業區內承租土地、建築物或國有房舍者土地租金緩繳1年或減收2成（二擇一），另協助承租購土地或廠房建築物者，其公共設施維護費得減半收取，並對已申請污水處理聯接使用廠商，使用費緩繳1年，分3年平均攤還，免計收滯納金及延遲利息，已經陸續開始受理申請，直接向當地工業區服務中心申請即可。
另外，加工區相關費用緩繳方面，經濟部也提供區內事業單位，土地租金緩繳1年或減收2成（二擇一）、展延投資建廠期限、污水處理費緩繳1年、營收衰退30%者，管理費緩繳1年等協助，另此4項紓困措施均免息，已開始受理申請。
為保住企業，經濟部加碼投入逾380億元，補貼服務業、製造業衰退逾五成艱困企業員工薪資及營運資金，其中企業營運資金的一次性補貼，共計67.3億元，服務業受惠家數13.1萬家，製造業4.4萬家， 可大大減輕雇主營運成本。
另外，經濟部表示，紓困2.0政策是針對營收大幅下滑五成，面臨裁員或有減薪之虞艱困企業規劃。針對下滑五成定義，有些企業擔心需一連2個月與去年同期比較，認條件嚴苛不敢提出，經濟部將親向產業界說明，不排除放寬紓困定義。
----------------------------------------------------------------------

[48] 股債齊跌　3月投信基金規模縮減

▲全球股價齊跌，投信基金規模連兩月縮減。（圖／資料照）
記者周康玉／台北報導
3月到肺炎疫情加重影響，全球股債齊跌，導致國內投信基金規模連續兩個月縮減，月減金額近2800億元。不過法人指出，在各國加強隔離管制、央行聯手救市下，疫情正逐漸改善、金融市場壓力緩

▲IMF下修預測台灣2020年經濟成長率至-4%。
國際貨幣基金 （IMF）於本週二（4月14日）按例於每年4月公佈年度首次《世界經濟展望報告》（World Economic Outlook），預估今（2020）年全球GDP將萎縮3%，恐將經歷自1930年大蕭條（Great Depression）以來最嚴重的衰退，衰退程度遠超過2008年的全球金融危機。
其中，美國經濟成長率今年將萎縮5.9%，中國大陸經濟成長率僅1.2%，歐元區今年經濟成長率萎縮7.5%；歐洲疫情最為嚴重的義大利和西班牙則分別萎縮9.1%和8%。而台灣經濟成長率由去年10月秋季報告的2%，下調6個百分點至負4%。但政府財經官員認為：「這根本是不可能的事情，參考就好」。
▲經濟預測也無法預期黑天鵝的到來。（圖／路透）
近年黑天鵝頻繁　經濟預測就好像在「擲筊」？
隨著計量經濟理論漸趨成熟，經濟預測早已實施多年，至於準不準更就另當別論了。然而，各家所用的經濟預測模型都建構在不同的假設基礎，所用的總體預測計量模型及影響變數也迴異。杜甫的《可嘆》詩云：「天上浮雲如白衣，斯須改變如蒼狗； 古往今來共一時，人生萬事無不有」，正反映出在瞬息萬變的全球化環境中，不僅地緣政治風險升溫，天災及傳染病等黑天鵝更無法放在模型中，以致經濟預測常常失準。
▲民間消費劇降，不少業者都在苦撐。（圖／記者周宸亘攝）
由支出面觀察台灣經濟成長　不難看出端倪
經濟預測模型相當複雜，非專業人士不易瞭解；但經濟成長該怎麼觀察倒不難。在總體經濟學的國民所得理論中，國內生產毛額就可由最終產品購買者支出面來估計，即GDP=C+I+G+X-M；C:指民間消費、I:投資毛額、G:政府消費支出、X:出口項、M為進口項。
就台灣GDP組成中，最大的二項分別為消費支出約占5成3，出口項約占6成6；換言之，只要觀察這二大項的變化，大概就可以洞察台灣經濟成長概況。
台灣雖沒有封城，而且此次有電商「宅經濟」加持，但實體面店卻因民眾不敢登門消費而大幅衰退；君不見，已陸續有許多知名的老字號餐廳關門，住宿及餐飲業哀鴻遍野。桃園機場旅運人數於4月14日全天僅669人，還不到去年平均每日13萬3,000人的旅運量的0.5%；第一航廈入境旅客人數更是掛零，創史上最低的旅運量新紀錄；沒有人潮就難有消費，更無錢潮，消費面慘狀可想而知。
此外，勞動部公布截至4月15日統計，台灣實

▲日圓。（圖／取自免費圖庫Pixabay）
記者吳靜君／台北報導
新冠肺炎（COVID-19）病毒觸發全球經濟活動大停擺，帶來經濟極大的危機，瑞銀今（17）日表示，疫情將可能在4月達到高峰，許多國家下旬將可能會陸續解除嚴格的防控措施，但是金融市場不確定性仍對以出口導向、大宗商品經濟體的亞洲貨幣造成壓力；惟人民幣及日圓則後勢看漲。
瑞銀認為全球新增確診病例將在4月到頂，許多國家4月下旬將陸續解除最嚴格的防況措施，但由於上半年惡劣的經商環境差不多已成定局，今年全球的經濟可能收縮0.5%到1%，其中中國大陸及亞洲（日本除外）的經濟分別放緩到1.5%與1.7%，是近年來少見的。
瑞銀財富管理投資總監辦公室外匯分析師陳得能表示，避險資金未來有一段時間依然偏好美元，除日圓之外，其他亞洲貨幣再第二季初期可能會繼續受壓。
陳得能表示，今年的情況與2008年的環境不能相提並論，各國的貨幣、財政政策比當時快速得多，值得借鑒的是，當時全球掀起巨浪，並迅速向全球市場蔓延，市場由最初面臨巨幅震盪到後來的小幅波動。
今年上半年，投資者還未能掌握經濟活動停擺的程度，更摸索不到下半年復甦的軌跡，但是金融市場難免波濤洶湧，陳得能指出，成熟市場極大部分的大部分的亞洲國家將在第二季收縮，在出口疲弱、投資人態度保守下，亞太貨幣顯得較為脆弱。在亞太地區，由出口導向、經常帳不太平衡或倚重大宗商品出口的經濟體，例如韓國、澳洲、紐西蘭、印度、印尼、馬來西亞、泰國及菲律賓等，其貨幣面臨較高風險。
中國大陸財政、貨幣政策雙管齊下　年底有望升至6.7元
至於中國大陸隨著第二季活動回向正軌，加上貨幣政策及財政政策雙管齊下，以抵禦第二季海外需求下降及產業鏈中斷帶來的經濟衝擊，為人民幣創造較穩定的環境。中國人行近日公布定向下調存款準備率100個基點，今年以來第三度降準，此外下調金融機構的超額存款準備利率，從0.72%下調到0.35%，為2008年以來首次。瑞銀也預期，中國大陸今年仍會再降準100到300個基本點，並下調中期貸款利率30到60個基本點，引導企業融資成本進一步下降。
在財政政策方面，預期中國大陸地方政府債券全年發行額比去年增加三分之一。同時美元漲勢可能在一至兩個月後無以為繼，支持人民幣及其他亞太貨幣的穩定性。瑞銀估計人民幣在6月底前持穩於7.0關口，年底前更升值至6.7。
此外，由於日本退休金資金持續流出，對第一

▲台積電2020年第一季法說會     。（圖／資料照）
記者周康玉／台北報導
在疫情衝擊下，台積電因7奈米市場規模放大量，帶動第一季業績再創創高，第二季則在遠距相關應用和5G持續布建帶動下，仍可望維持高檔，ETtoday新聞雲針對第一季業績、第2季展望、全年半導體產業及公司年度展望、對客戶需求及產業趨勢看法、全年資本支出調整、5奈米及3奈米量產進度、美國持續打壓華為等七大重點整理一次看。
受法說會激勵，台積電ADR周四收52.40美元，大漲2.74美元或5.52%，ADR收盤價折合新台幣每股約315.65元。
▼台積電1Q20法說會重點一次看，圖點開放大。（記者周康玉製表）
----------------------------------------------------------------------

[78] 下周汽油擬降0.9元　九二無鉛每公升17元創史上新低

▲新冠肺炎（COVID-19）疫情延燒歐美，專家預估下周國內汽柴油每公升零售價將各降0.9元。（圖／記者姜國輝攝）
記者林淑慧／台北報導
國際油價直直落，專家預估下周國內汽柴油每公升零售價將各降0.9元及1元，影響所及，九二無鉛汽油每公升將跌至17元，再創史上新低價紀錄，提醒想加油的民眾不妨再等等，才能替自己省點荷包。
依據中油目前油價估算，調整後的參考零售價格分別為92無鉛汽油為17元、95無鉛汽油18.5元、98無鉛20.5元、超級柴油14元。
油價專家表示，新冠肺炎（COVID-19）疫情延燒歐美，衝擊全球原油需求，牽動國際油價持續走跌；累算至4月16日之7D3B週均價為20.34美元，較上週23.77下跌3.43美元。新臺幣兌美元匯率30.086較上週30.160升值0.074元。
----------------------------------------------------------------------

[79] 創8年新高！勞動部統計「1.4萬人放無薪假」　竹科也暴增10倍

圖文／CTWANT
新冠肺炎影響，許多觀光業及餐飲業大受打擊。勞動部公布截至4月15日止，台灣實施無薪假企業有588家，影響勞工1.4萬多人，一周暴增220家，共6262人，增加速度快，無薪假人數續創8年新高。除了住宿餐飲及旅行社業者為重災區外，無薪假也波及到新竹科學園區，一周以來家數